### **Chain**

Building up a simple **chain** that combines 4 key concepts:
* Using chat messages in our graph
* Using chat models
* Biding tools to our LLM
* Executing tool calls in our graph

##### **Messages**

Chat models can use `messages`, which capture different roles within a conversation.

LangChain supports various messages types, including `HumanMessage`, `AIMessage`, `SystemMessage`, and `ToolMessage`.

These represent a message from user, chat model, for the chat model to instruct behavior, and from a tool call.

Let's create a list of messages. Each message can be supplied with a few things:

* `content` - content of the message
* `name` - optionally, who is creating the message
* `response_metadata` - optionally, a dict of metadata that is often specific to each model provider

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

messages = [AIMessage(content=f'So you said you were researching ocean mammals?', name='Model')]
messages.extend([HumanMessage(content=f"Yes, that's right.", name='Lance')])
messages.extend([AIMessage(content=f'Great, what would you like to learn aboyut?', name='Model')])
messages.extend([HumanMessage(content=f'I want to learn about the best places to see Orcas in the US.', name='Lance')])

for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

So you said you were researching ocean mammals?
================================ Human Message =================================
Name: Lance

Yes, that's right.
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn aboyut?
================================ Human Message =================================
Name: Lance

I want to learn about the best places to see Orcas in the US.


##### **Chat Models**

Chat models can use a sequence of message as input and support message roles.

There are many to choose from!

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')
result = llm.invoke(messages)
type(result)

langchain_core.messages.ai.AIMessage

In [12]:
result.pretty_print()

================================== Ai Message ==================================

Orcas, also known as killer whales, can be seen in several locations across the U.S. Here are some of the best places to spot them:

1. **Seattle and the San Juan Islands, Washington**: This region is one of the best places in the world to see orcas. The San Juan Islands are home to resident pods of orcas, particularly the Southern Resident killer whales. Various charter services offer whale-watching tours in the area.

2. **Vancouver Island, British Columbia**: While it's technically in Canada, it's very accessible from the U.S., especially from Washington State. The waters around Victoria, British Columbia, are renowned for orca sightings.

3. **California Coast**: While not as consistent as Washington, orcas can sometimes be seen offshore along the California coast, particularly around Monterey Bay. In winter and spring, transient orcas following their prey can be spotted.

4. **Alaska**: Orcas can be 

In [13]:
result.response_metadata

{'token_usage': {'completion_tokens': 330,
  'prompt_tokens': 69,
  'total_tokens': 399,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_64e0ac9789',
 'id': 'chatcmpl-BLFMS1bZkbzap1gNnApWN45AEYowW',
 'finish_reason': 'stop',
 'logprobs': None}

##### **Tools**

Tools are need whenever you want a model to control parts of your code or call out to external APIs.

Many LLMs providers supports tool calling.

The tool calling interface in LangChain is simple.

You can pass any Python function into `ChatModel.bind_tools()`.

In [14]:
def multiply(a:int, b:int) -> int:
    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [17]:
tool_call = llm_with_tools.invoke([HumanMessage(content="What is 2 multipled by 3", name="Lucas")])
tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XLx3I3P51x7jSxXMSC1jGGfK', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 51, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BLFUzITBmdoovSRRyUO0VCIPI3NZM', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2eec0f5d-fe36-41ee-8340-91d15ec5e838-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_XLx3I3P51x7jSxXMSC1jGGfK', 'type': 'tool_call'}], usage_metadata={'input_tokens': 51, 'output_tokens': 18, 'total_tokens': 69, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [18]:
tool_call.additional_kwargs['tool_calls']

[{'id': 'call_XLx3I3P51x7jSxXMSC1jGGfK',
  'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'},
  'type': 'function'}]

##### **Using messages as state**

With these foundations in place, we can now use `messages` in our graph state.

Let's define our state `MessagesState`.

It's defined as a `TypedDict` with a single key: `messages`.

`messages` is simply a list of type `AntMessage`, meaning it's a list of messages.

In [19]:
from typing import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

##### **Reducers**

Now, we have a minor problem!

As graph runs, we want to append messages to our `messages` state key.

But each node will also override the prior state value.

**Reducer functions** address this.

They allow us to specify how state updates are performed.

If no reducer function is explcitly specifeid, then it is assumed that all updates to that key should *override* it.

Since we want to apped messages, we can user a pre-built `add_messages` reducer!

This ensures that state updates you send to the graph are appended to the existing list of messages.

We annotate (via `Annotated`) our key with a reducer function as metadata.

In [20]:
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

Since habing a list of messages in your state is so common, LangGraph has a pre-built `MessagesState`!

`MessagesState` is defined:

* With a pre-build single `messages` key
* Which is a list of `AnyMessage` objects and uses the `add_messages` reducer

In [21]:
from langgraph.graph import MessagesState

class State(MessagesState):
    # add any keys needed beyond messages, which is pre-built
    pass

The `MessagesState` and `State` both work equivalently!

To go a bit deeper, we can see how the `add_messages` reducer works in isolation.

In [23]:
# initial state
initial_messages = [AIMessage(content='Hello! How can I assist you?', name='Model'),
                    HumanMessage(content="I'm looking for information on marine biology.", name='Lucas')]

# new messsage to add
new_message = AIMessage(content='Sure, I can help with that. What specifically are you interested in?', name='Model')

# test
add_messages(initial_messages, new_message)

[AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}, name='Model', id='42cdb7e5-c850-4713-b871-08ec3d221b02'),
 HumanMessage(content="I'm looking for information on marine biology.", additional_kwargs={}, response_metadata={}, name='Lucas', id='07e195f4-b89c-4e80-accf-30e7356f7106'),
 AIMessage(content='Sure, I can help with that. What specifically are you interested in?', additional_kwargs={}, response_metadata={}, name='Model', id='e6b98bc3-ef6b-44bd-96db-55ab550b9a24')]

##### **Our graph**

Now, let's use `MessagesState` with a graph